In [2]:
from elasticsearch import Elasticsearch
import requests

ES_HOST = "http://orca-test:9200"

es = Elasticsearch(ES_HOST)



In [ ]:

mappings = {
    "properties": {
        "title": {
            "type": "text",    
            "analyzer": "german"
        },
        "text": {
            "type": "text",    
            "analyzer": "german"
        },
        "year": {
            "type": "integer"
        },
    }
}

es.indices.create(index="resolutions", mappings=mappings)

In [ ]:
# Get all resolutions from api-nuxt-beschlusswiki.vercel.app/resolution

resolutions = requests.get("https://nuxt-beschlusswiki-api.vercel.app/resolution").json()


for resolution in resolutions:
    if resolution.get("state") != "live": break

    doc = {
        "title": resolution.get("body").get("title"),
        "text": resolution.get("body").get("text"),
        "year": resolution.get("body").get("year"),
    }
    es.index(index="resolutions", id=resolution.get("_id"), document=doc)
    print("Added resolution with id {} to index resolutions".format(resolution.get("_id")))

In [ ]:
# List Number of indexed resolutions
es.indices.refresh(index="resolutions")
es.cat.count(index="resolutions", params={"format": "json"})

In [34]:
search_term = "Stadt prägen"

def displayHit(hit):
    print("[{}] - Score: {} \t ({})".format(hit.get("_id"), hit.get("_score"), hit.get("_source").get("title")))

# Elasticsearch-Query erstellen
query = {
    "query": {
        "bool": {
            "should": [
                {
                    "multi_match": {
                        "query": search_term,
                        "fields": ["title^2", "text"],
                        "fuzziness": "AUTO",
                        "boost": 1.5
                    }
                },
                {
                    "match_phrase": {
                        "text": {
                            "query": search_term,
                            "slop": 1,
                        }
                    }
                },
                {
                    "span_near": {
                        "clauses": [
                            {
                                "span_term": {
                                    "text": {
                                        "value": "Stadt"
                                    }
                                }
                            },
                            {
                                "span_term": {
                                    "text": {
                                        "value": "prägen"
                                    }
                                }
                            }
                            # Weitere Klauseln für weitere Wörter
                        ],
                        "slop": 0,  # Anzahl der erlaubten Wörter zwischen den Ausdrücken (0 für aufeinander folgende Wörter)
                        "in_order": True,
                        "boost": 2
                    }
                }
            ]
        }
    }
}


# Die Suche ausführen
res = es.search(index="resolutions", body=query)
print ("Found %d documents:" % res['hits']['total']['value'])
for doc in res['hits']['hits']:
    displayHit(doc)

Found 46 documents:
[6519ba46510e26b87f84d77e] - Score: 10.357319 	 (Stadt, Land, Struktur verändern. Das gute Leben für alle)
[6519ba46510e26b87f84d78a] - Score: 6.7918434 	 (Sozial 4.0 – Unsere Zukunft: frei, gerecht und digital)
[6519ba46510e26b87f84d787] - Score: 6.5933876 	 (Zukunft statt Grenzen – Wir investieren in die Integrati-)
[6519ba46510e26b87f84d79f] - Score: 6.5933876 	 (Echtes Ankommen statt rechter Hetze)
[6519ba46510e26b87f84d778] - Score: 5.9474516 	 (Unsere Jugendwahlkampagne 2017)
[6519ba46510e26b87f84d79b] - Score: 5.920248 	 (Jin, Jiyan, Azadi! Solidarität mit den Protesten im)
[6519ba46510e26b87f84d7ab] - Score: 5.8144407 	 (Zeitenwende – aber richtig!)
[6519ba46510e26b87f84d79d] - Score: 5.6971693 	 (Volle Kraft voraus! – Zeitenwende auch in der)
[6519ba46510e26b87f84d788] - Score: 5.517683 	 (Steuern und Gerechtigkeit)
[6519ba46510e26b87f84d79e] - Score: 4.6901617 	 (Wir sind solidarisch. Komme, was wolle.)


C:\Users\Chris\AppData\Local\Temp\ipykernel_25736\3285134854.py:58: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res = es.search(index="resolutions", body=query)
